### 03 - HEAT STRESS OUTPUTS EXPLORATION

#### 03.1 - Libraries import

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### 03.2 - Temporal patterns analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ========== PARAMETRI MODIFICABILI ==========
csv_path = "heatstress_all_timestamps_all_years.csv"
metodo_statistico = "95 Perc."  # Opzioni: "Media", "Mediana", "95 Perc.", "99 Perc.", "Massimo"
min_soglie_superate = 4         # Minimo numero di indici oltre soglia

# ========== CARICAMENTO ==========
df = pd.read_csv(csv_path, parse_dates=["Timestamp"])
df["Anno"] = df["Timestamp"].dt.year

# Adatta i nomi delle colonne (rimuove simboli speciali)
df.columns = df.columns.str.replace("°", "").str.replace("(C)", "").str.strip()

# Colonna confronto e soglia
col_valore = metodo_statistico
col_soglia = "Soglia (C)"
col_confronto = f"{metodo_statistico} > Soglia"

# Verifica che esista
if col_confronto not in df.columns:
    raise ValueError(f"Colonna '{col_confronto}' non trovata. Colonne disponibili:\n{df.columns.tolist()}")

# ========== COSTRUZIONE GIORNI PERICOLOSI ==========
giorni_pericolo = []

for giorno, gruppo in df.groupby("Timestamp"):
    sotto_df = gruppo.set_index("Indice")
    bool_superata = sotto_df[col_confronto] == "si"
    n_superata = bool_superata.sum()
    
    if n_superata >= min_soglie_superate:
        residui = (sotto_df.loc[bool_superata, col_valore] - sotto_df.loc[bool_superata, col_soglia]).sum()
        giorni_pericolo.append({
            "Data": giorno,
            "Anno": giorno.year,
            "n_superata": n_superata,
            "residuo_totale": residui
        })

# ========== AGGREGAZIONE PER ANNO ==========
df_pericolo = pd.DataFrame(giorni_pericolo)
summary = df_pericolo.groupby("Anno").agg({
    "Data": "count",
    "residuo_totale": "sum"
}).rename(columns={"Data": "giorni_pericolosi"})

summary["intensità_media"] = summary["residuo_totale"] / summary["giorni_pericolosi"]

# ========== GRAFICO ==========
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.bar(summary.index, summary["giorni_pericolosi"], color="orange", label="Giorni oltre soglia")
ax1.set_ylabel("Giorni oltre soglia", color="orange")
ax1.set_xlabel("Anno")
ax1.tick_params(axis='y', labelcolor="orange")

ax2 = ax1.twinx()
ax2.plot(summary.index, summary["intensità_media"], color="blue", marker='o', label="Intensità media")
ax2.set_ylabel("Intensità cumulativa media", color="blue")
ax2.tick_params(axis='y', labelcolor="blue")

plt.title(f"Giorni pericolosi (≥ {min_soglie_superate} indici oltre soglia) - Metodo: {col_valore}")
fig.tight_layout()
plt.grid(True)
plt.show()